In [1]:
# %matplotlib 
import json
import datetime
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import urllib, json

In [2]:
API_KEY = ''
meetup_group = 'central_london_data_science'
max_event_count = 100
max_per_page = 200

In [3]:
def get_json_from_meetup_api(url):
    full_url = url + '&key=' + API_KEY
    response = urllib.urlopen(full_url)
    return json.loads(response.read())

def get_members(group_name, offset = 0):
    print group_name, offset
    url = 'https://api.meetup.com/2/members?&sign=true&photo-host=public&group_urlname='+group_name+'offset='+str(offset)+'&page='+str(max_per_page)
    url = 'https://api.meetup.com/2/members?offset='+str(offset)+'&format=json&group_urlname='+group_name+'&photo-host=public&page='+str(max_per_page)+'&order=name&sig_id=188066678&sig=66c1ac0920cc0731d877d64d01ca0ff1b6c7e3fb'
    return get_json_from_meetup_api(url)

def get_events(group_name):
    url = 'https://api.meetup.com/2/events?&sign=true&photo-host=public&group_urlname='+group_name+'&status=past,upcoming&page='+str(max_event_count)
    return get_json_from_meetup_api(url)

def get_group_info(group_name):
    url = 'https://api.meetup.com/'+group_name+'?&sign=true&photo-host=public'
    return get_json_from_meetup_api(url)

def get_all_members(group_name, count):
    loops = count/max_per_page
    extra_loop = count%max_per_page is not 0
    tota_loops = loops + extra_loop
    print tota_loops
    store_members = []
    for i in range(tota_loops):
        current_get = get_members(group_name, offset = i)
        print current_get
        store_members += current_get['results']
    return store_members

def get_epoc_in_days_since(start_epoc, end_epoc):
    start = datetime.datetime.fromtimestamp(start_epoc/1000)
    end = datetime.datetime.fromtimestamp(end_epoc/1000)
    return (end - start).days

In [4]:
group_info = get_group_info(meetup_group)

In [5]:
group_info.keys()

[u'lat',
 u'timezone',
 u'organizer',
 u'id',
 u'category',
 u'city',
 u'lon',
 u'state',
 u'description',
 u'join_mode',
 u'who',
 u'visibility',
 u'photos',
 u'group_photo',
 u'link',
 u'members',
 u'next_event',
 u'urlname',
 u'name',
 u'key_photo',
 u'created',
 u'country',
 u'localized_country_name']

In [6]:
group_start =  datetime.datetime.fromtimestamp(group_info['created']/1000)
group_start

datetime.datetime(2016, 4, 20, 2, 19, 11)

In [7]:
group_age = (datetime.datetime.now() -  group_start).days
group_age

220

In [8]:
group_info['members']

416

In [9]:
all_member_data = get_all_members(meetup_group, group_info['members'])

3
central_london_data_science 0
{u'meta': {u'count': 200, u'updated': 1480169545000, u'description': u'API method for accessing members of Meetup Groups', u'title': u'Meetup Members v2', u'url': u'https://api.meetup.com/2/members?offset=0&format=json&group_urlname=central_london_data_science&sig=66c1ac0920cc0731d877d64d01ca0ff1b6c7e3fb&photo-host=public&page=200&sig_id=188066678&key=424d2555153e34a47167131498230&order=name', u'total_count': 416, u'lon': u'', u'next': u'https://api.meetup.com/2/members?offset=1&format=json&group_urlname=central_london_data_science&sig=66c1ac0920cc0731d877d64d01ca0ff1b6c7e3fb&photo-host=public&page=200&sig_id=188066678&key=424d2555153e34a47167131498230&order=name', u'lat': u'', u'link': u'https://api.meetup.com/2/members', u'id': u'', u'method': u'Members'}, u'results': [{u'status': u'active', u'city': u'London', u'name': u'Aamer', u'other_services': {}, u'country': u'gb', u'topics': [{u'name': u'Cloud Computing', u'urlkey': u'cloud-computing', u'id': 15

In [10]:
len(all_member_data)

416

In [11]:
all_member_data[0]['joined']

1470119142000

In [12]:
group_info['members']

416

In [13]:
# get date joined in days resolution
dateJoined = [x["joined"]/1000 for x in all_member_data if 'joined' in x.keys()]

In [14]:
len(dateJoined)

415

In [15]:
epochs = sorted(dateJoined)
when_people_joined = [datetime.datetime.fromtimestamp(x) for x in epochs]

In [16]:
len(sorted(when_people_joined))

415

In [17]:
running_total_of_members = []
for i in range(group_age):
    sum_to_day = len([j for j in when_people_joined if (datetime.datetime.now()-j).days <= i])
    running_total_of_members.append(sum_to_day)

In [21]:
eventData = get_events(meetup_group)['results']

In [22]:
event_count = len(eventData)
event_count

17

In [23]:
create_event_date = [datetime.datetime.fromtimestamp(x['created']/1000) for x in eventData]
start_event_date = [datetime.datetime.fromtimestamp(x['time']/1000) for x in eventData]

create_event_day_from_group_start = [(x-group_start).days  for x in create_event_date]
start_event_day_from_group_start = [(x-group_start).days  for x in start_event_date]

len(create_event_day_from_group_start),len(start_event_day_from_group_start)

(17, 17)

In [ ]:
plt.plot(range(group_age),running_total_of_members,color='g')
for i in create_event_day_from_group_start:
    plt.axvline(x=i,color='r', linewidth=0.3)
for i in start_event_day_from_group_start:
    plt.axvline(x=i,color='b', linewidth=0.3)

In [ ]:
members = mpatches.Patch(color='g', label='No. of members')
event_anounce = mpatches.Patch(color='r', label='Event anounced')
event_start = mpatches.Patch(color='b', label='Event start')

plt.legend(handles=[members,event_anounce,event_start],loc=4)
plt.show()